In [1]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow import keras


def load(data_path):
    with open(data_path,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

class MnistData:
    def __init__(self, filenames, need_shuffle, datatype='training'):
        all_data = []
        all_labels = []
        x_train, y_train, x_test, y_test = load(filenames) #"data/mnist.pkl"
        
        if datatype=='training':
            self._data = x_train / 127.5 -1
            self._labels = y_train
            print(self._data.shape)
            print(self._labels.shape)
        else:
            self._data = x_test / 127.5 -1
            self._labels = y_test
            print(self._data.shape)
            print(self._labels.shape)
            
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

In [2]:
data = "../4_basic_image_recognition/data/mnist.pkl"
train_data = MnistData(data, True, 'training')
test_data = MnistData(data, False, 'test')

(60000, 784)
(60000,)
(10000, 784)
(10000,)


## Tensorboard Embedding
<br>
https://www.tensorflow.org/guide/embedding

In [3]:
# define a function for tensorboard data visualization

def variable_summaries(name_scope, var):
    with tf.name_scope('summary_' + name_scope):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [4]:
from tensorflow.contrib.tensorboard.plugins import projector
import os


DIR = os.getcwd()
num_images = 3000
embedding_img = tf.stack(test_data._data[:num_images])
# loading 3000 pics from 10.000 test_data
embedding = tf.Variable(embedding_img, trainable=False, name='embedding')

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28*28], name='x-intput')
    y = tf.placeholder(tf.int64, [None], name='y-input')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x-input-image')

# for tensorboard image visualization
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1]) # see x.shape
    tf.summary.image('input', image_shaped_input, 10)
    
with tf.name_scope('layers'):
    conv_1 = tf.layers.conv2d(inputs=x_image, 
                              filters=32, 
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=tf.nn.relu,
                              name= 'conv1')
    """
    # create summary in tensorboard
    conv_1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, conv_1.name)
    variable_summaries('conv1_kernel', conv_1_vars[0])
    variable_summaries('conv1_bias', conv_1_vars[1])
    """

    pool1 = tf.layers.max_pooling2d(inputs=conv_1,
                                    pool_size=(2, 2),
                                    strides=(2,2),
                                    name='pool1')

    conv_2 = tf.layers.conv2d(inputs=pool1,
                              filters=64,
                              kernel_size=(5, 5),
                              padding = 'same',
                              activation=tf.nn.relu,
                              name='conv2')

    pool2 = tf.layers.max_pooling2d(inputs=conv_2,
                                    pool_size=(2,2),
                                    strides=(2,2),
                                    name= 'pool2')

    # fc layer1
    flatten = tf.layers.flatten(pool2, name= 'fc1_flatten')
    # fc layer2
    y_ = tf.layers.dense(flatten, 10, name= 'fc2_flatten')
    
    # create summary in tensorboard
    fc2_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'fc2_flatten')
    variable_summaries('fc2_kernel', fc2_vars[0])
    variable_summaries('fc2_bias', fc2_vars[1])

with tf.name_scope('loss'):
    #sparse_softmax_cross_entropy include one_hot encoding
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
    tf.summary.scalar('loss', loss)


with tf.name_scope('accuracy'):
    
    predict = tf.argmax(y_, 1)
    correct_prediction = tf.equal(predict, y)
    """
    with tf.name_scope('correct_prediction'):
        tf.summary.scalar('correct_prediction', correct_prediction)
    """
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    with tf.name_scope('accuracy'):
        tf.summary.scalar('accuracy', accuracy)

    
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 50


# train 10k: %
with tf.Session() as sess:
    sess.run(init)
    
    ###############################################################################
    # also for tensorboard visualization -> stolen things, but works!
    if tf.gfile.Exists(DIR + '/projector/projector/metadata.tsv'):
        tf.gfile.DeleteRecursively(DIR + '/projector/projector/metadata.tsv')
        
    with open(DIR + '/projector/projector/metadata.tsv', 'w') as f:
        y_one_hot = tf.one_hot(test_data._labels[:], 10, dtype=tf.float32)
        labels = sess.run(tf.arg_max(y_one_hot, 1))
        #labels = sess.run(tf.arg_max(test_data._labels[:], 1))
        for i in range(num_images):
            f.write(str(labels[i]) + '\n')

    # merge summary
    merged = tf.summary.merge_all()    
    # setup projector for plotting
    projector_writer = tf.summary.FileWriter(DIR + '/projector/projector', sess.graph)
    saver = tf.train.Saver()
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = embedding.name
    embed.metadata_path = DIR + '/projector/projector/metadata.tsv'
    print('metadata_path: ' + embed.metadata_path)
    embed.sprite.image_path = DIR + '/projector/data/mnist_10k_sprite.png'
    embed.sprite.single_image_dim.extend([28, 28])
    projector.visualize_embeddings(projector_writer, config)

    ###############################################################################
    
    # start training
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        ###########################################################################
        # standard params for projector
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
                
        summary, loss_val, acc_val, _ = sess.run([merged, loss, accuracy, train_op], 
                                                 feed_dict={x: batch_data, y: batch_labels},
                                                 options=run_options,
                                                 run_metadata=run_metadata)
        
        projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        projector_writer.add_summary(summary, i)
        ###########################################################################
                
        # print train kpi
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        
        # perform test and test kpi
        if (i+1) % 1000 == 0:
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict = {x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))
    
    saver.save(sess, DIR + '/projector/projector/model.ckpt', global_step=train_steps)
    projector_writer.close()
    
# start tensorboard with: tensorboard --logdir=projector/projector

Instructions for updating:
Use `tf.math.argmax` instead
metadata_path: /home/bai/PycharmProjects/tensorflow_for_enginner/6_optimazation/projector/projector/metadata.tsv
[Train] Step: 100, loss: 1.67580, acc: 0.65000
[Train] Step: 200, loss: 0.67392, acc: 0.90000
[Train] Step: 300, loss: 0.78551, acc: 0.80000
[Train] Step: 400, loss: 0.66212, acc: 0.70000
[Train] Step: 500, loss: 0.18241, acc: 1.00000
[Train] Step: 600, loss: 0.10630, acc: 1.00000
[Train] Step: 700, loss: 0.32330, acc: 0.90000
[Train] Step: 800, loss: 0.21666, acc: 0.95000
[Train] Step: 900, loss: 0.22357, acc: 0.90000
[Train] Step: 1000, loss: 0.08862, acc: 1.00000
[Test ] Step: 1000, acc: 0.93500
